In [1]:
import pandas as pd
import numpy as np
from line_profiler import LineProfiler

%load_ext Cython

In [2]:
# https://www.kaggle.com/apratim87/housingdata
data = pd.read_csv('housingdata.csv', header=None)

In [3]:
data = (data - data.mean()) / data.std()

In [4]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()

In [5]:
X = data.values[:, :-1]
y = data.values[:, -1:]

У нас есть выборка $X \in R^{n\times d}, y\in R^n$, состоящая из $n$ объектов. Каждый из $n$ объектов описывается вектором из $d$ признаков (строка матрицы $X$) и для каждого объекта мы знаем значение целевой переменной $y$. В данной задаче по некоторым параметрам мы хотим научиться предсказывать стоимость квадратного метра жилья.


Предположим, что для объекта $i$ мы можем описать $y_i$ линейной комбинацией $x_i$ с некоторыми весами $w$, где w, x_i - вектора размера $d \times 1$, а $y$ - вещественное число.
$$y_i \sim <w,x_i>$$


Методом наименьших квадратов найдем веса $w$.
$$ J(w) = \frac{1}{n} \sum_{i=1}^n (<w,x_i> - y_i)^2$$
$$ J(w) \rightarrow \min_w $$

Минимум данного функционала будем искать методом градиентного спуска (те будет идти по направлению противоположному градиенту):
$$ w_j = w_j - \alpha \frac{\partial}{\partial w_j}J(w) $$
$$ \frac{\partial}{\partial w_j}J(w)  = \frac{2}{n}  \sum_{i=1}^n (<w, x_i> - y_i)\cdot x_i^j $$

In [6]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr = lr.fit(X, y)
print ((y - lr.predict(X)) ** 2).mean()

0.258844771986


In [7]:
N_ITER = 300

In [8]:
def gradient_decent_python_naive(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        for f in xrange(n_features):
            gradient = 0
            for obj in xrange(n_objects):
                gradient += ((X[obj, :] * w).sum() - y[obj, 0]) * X[obj, f] 
            gradient = gradient * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient
    return w.reshape(-1, 1)

In [9]:
w = gradient_decent_python_naive(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.25884485836


In [55]:
%%timeit
gradient_decent_python_naive(X, y, n_iter=N_ITER)

1 loop, best of 3: 10.3 s per loop


In [12]:
profile_print(gradient_decent_python_naive, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 13.6046 s
File: <ipython-input-8-4afa2296c546>
Function: gradient_decent_python_naive at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def gradient_decent_python_naive(X, y, n_iter=100, alpha=0.1):
     2         1            4      4.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     3         1           16     16.0      0.0      w = np.random.rand(n_features)
     4         1            8      8.0      0.0      w_old = np.random.rand(n_features)
     5       301          193      0.6      0.0      for iteration in xrange(n_iter):
     6       300         1234      4.1      0.0          np.copyto(w_old, w)
     7      4200         3550      0.8      0.0          for f in xrange(n_features):
     8      3900         2173      0.6      0.0              gradient = 0
     9   1977300      1248516      0.6      9.2              for obj in xrange(n_objects):
    10   

In [13]:
#  поменяли порядок циклов по объектам и признакам

def gradient_decent_python(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            for f in xrange(n_features):
                gradient[f] += ((X[obj, :] * w).sum() - y[obj, 0]) * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [14]:
w = gradient_decent_python(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258844773377


In [56]:
%%timeit
gradient_decent_python(X, y, n_iter=N_ITER)

1 loop, best of 3: 10.1 s per loop


In [16]:
profile_print(gradient_decent_python, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 14.3494 s
File: <ipython-input-13-265692020db8>
Function: gradient_decent_python at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_python(X, y, n_iter=100, alpha=0.1):
     4         1            4      4.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           17     17.0      0.0      w = np.random.rand(n_features)
     6         1            8      8.0      0.0      w_old = np.random.rand(n_features)
     7       301          206      0.7      0.0      for iteration in xrange(n_iter):
     8       300         1127      3.8      0.0          np.copyto(w_old, w)
     9       300         1586      5.3      0.0          gradient = np.zeros(n_features)
    10    152100        86705      0.6      0.6          for obj in xrange(n_objects):
    11   2125200      1389304      0.7      9.7              for f in xrange(n_features):
    1

In [17]:
#  вынесем подсчет ошибки

def gradient_decent_python_faster(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]  # changed
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [18]:
w = gradient_decent_python_faster(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.25887270555


In [19]:
%%timeit
gradient_decent_python_faster(X, y, n_iter=N_ITER)

1 loop, best of 3: 2.09 s per loop


In [20]:
profile_print(gradient_decent_python_faster, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 4.54663 s
File: <ipython-input-17-37a8e8495c53>
Function: gradient_decent_python_faster at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_python_faster(X, y, n_iter=100, alpha=0.1):
     4         1            5      5.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           23     23.0      0.0      w = np.random.rand(n_features)
     6         1           11     11.0      0.0      w_old = np.random.rand(n_features)
     7       301          233      0.8      0.0      for iteration in xrange(n_iter):
     8       300         1257      4.2      0.0          np.copyto(w_old, w)
     9       300         1660      5.5      0.0          gradient = np.zeros(n_features)
    10    152100        85690      0.6      1.9          for obj in xrange(n_objects):
    11    151800      1108333      7.3     24.4              diff = (X[obj, :] * 

In [21]:
from numba import jit

In [22]:
# numba !!!

@jit(nopython=True)
def gradient_decent_numba(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):  # changed
            w_old[f] = w[f]
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [23]:
w = gradient_decent_numba(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.25884648855


In [24]:
%%timeit
gradient_decent_numba(X, y, n_iter=N_ITER)

10 loops, best of 3: 31.9 ms per loop


In [25]:
# уберем цикл по объектам

def gradient_decent_numpy(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    w_old = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros((n_features, 1))  # chaned
        diff = X.dot(w) - y
        for f in xrange(n_features):
            gradient[f, 0] = ((X.dot(w) - y) * X[:, f:f+1]).sum()
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w  # changed

In [26]:
w = gradient_decent_numpy(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258851799109


In [27]:
%%timeit
gradient_decent_numpy(X, y, n_iter=N_ITER)

10 loops, best of 3: 55.9 ms per loop


In [28]:
profile_print(gradient_decent_numpy, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 0.080551 s
File: <ipython-input-25-c4a9874affe5>
Function: gradient_decent_numpy at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_numpy(X, y, n_iter=100, alpha=0.1):
     4         1            5      5.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           36     36.0      0.0      w = np.random.rand(n_features, 1)  # changed
     6         1           11     11.0      0.0      w_old = np.random.rand(n_features, 1)  # changed
     7       301          204      0.7      0.3      for iteration in xrange(n_iter):
     8       300          609      2.0      0.8          np.copyto(w_old, w)
     9       300          587      2.0      0.7          gradient = np.zeros((n_features, 1))  # chaned
    10       300         3018     10.1      3.7          diff = X.dot(w) - y
    11      4200         3363      0.8      4.2          for f 

In [29]:
# уберем еще и цикл по признакам

def gradient_decent_numpy_faster(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [30]:
w = gradient_decent_numpy_faster(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258846693195


In [31]:
%%timeit
gradient_decent_numpy_faster(X, y, n_iter=N_ITER)

100 loops, best of 3: 9.64 ms per loop


In [32]:
profile_print(gradient_decent_numpy_faster, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 0.02134 s
File: <ipython-input-29-93cc6efbbf13>
Function: gradient_decent_numpy_faster at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_numpy_faster(X, y, n_iter=100, alpha=0.1):
     4         1            4      4.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           19     19.0      0.1      w = np.random.rand(n_features, 1)  # changed
     6       301          313      1.0      1.5      for iteration in xrange(n_iter):
     7       300        16787     56.0     78.7          gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
     8       300         2988     10.0     14.0          gradient = gradient * 2 / n_objects
     9       300         1228      4.1      5.8          w -= alpha * gradient
    10         1            1      1.0      0.0      return w



In [33]:
@jit(nopython=True)
def gradient_decent_numpy_faster_numba(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [34]:
# # самая большая проблема - sum c axis

# w = gradient_decent_numpy_faster_numba(X, y)
# print ((y - X.dot(w)) ** 2).mean()

In [35]:
%%cython
import numpy as np
cimport numpy as np

def gradient_decent_numpy_faster_cython(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [36]:
w = gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.25884564821


In [37]:
%%timeit
gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)

100 loops, best of 3: 9.1 ms per loop


In [38]:
%%cython -a
import numpy as np
cimport numpy as np

def gradient_decent_numpy_faster_cython(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [39]:
# добавим типы

In [40]:
%%cython -a
import numpy as np
cimport numpy as np

cpdef gradient_decent_numpy_faster_cython(np.ndarray[np.float64_t, ndim=2] X, 
                                          np.ndarray[np.float64_t, ndim=2] y, 
                                          int n_iter=100, 
                                          np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.ndarray[np.float64_t, ndim=2] w;
    cdef np.ndarray[np.float64_t, ndim=2] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [41]:
w = gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258847793085


In [42]:
%%timeit
gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)

100 loops, best of 3: 9.83 ms per loop


In [43]:
#  вернемся к нашему хорошему коду на питоне и добавим типы

In [44]:
%%cython -a
import numpy as np
cimport numpy as np


cpdef gradient_decent_python_faster_cython_v0(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [45]:
w = gradient_decent_python_faster_cython_v0(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258856237834


In [46]:
%%timeit
res = gradient_decent_python_faster_cython_v0(X, y, n_iter=N_ITER)

1 loop, best of 3: 684 ms per loop


In [47]:
# заменим код, где много вызовов к python (отмечены желтым)

In [48]:
%%cython -a
import numpy as np
cimport numpy as np


cpdef gradient_decent_python_faster_cython_v1(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.float64_t diff
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    gradient = np.zeros(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):
            gradient[f] = 0
            w_old[f] = w[f]
        for obj in xrange(n_objects):
            diff = - y[obj, 0]
            for f in xrange(n_features):
                diff += X[obj, f] * w[f]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        for f in xrange(n_features):
            gradient[f] = gradient[f] * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient[f]
    return w.reshape(-1, 1)

In [49]:
w = gradient_decent_python_faster_cython_v1(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258846823549


In [50]:
%%timeit
res = gradient_decent_python_faster_cython_v1(X, y, n_iter=N_ITER)

100 loops, best of 3: 5.11 ms per loop


In [51]:
# добавим декораторы

In [52]:
%%cython -a
import numpy as np
cimport numpy as np
import cython


@cython.boundscheck(False)
@cython.cdivision(True)
cpdef gradient_decent_python_faster_cython_v2(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.float64_t diff
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    gradient = np.zeros(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):
            gradient[f] = 0
            w_old[f] = w[f]
        for obj in xrange(n_objects):
            diff = - y[obj, 0]
            for f in xrange(n_features):
                diff += X[obj, f] * w[f]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        for f in xrange(n_features):
            gradient[f] = gradient[f] * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient[f]
    return w.reshape(-1, 1)

In [53]:
w = gradient_decent_python_faster_cython_v2(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.25885518199


In [54]:
%%timeit
res = gradient_decent_python_faster_cython_v2(X, y, n_iter=N_ITER)

100 loops, best of 3: 4.24 ms per loop
